In [1]:
import pandas as pd
import glob
import ssqueezepy #para convertir la señal en imagen
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import os
import mne #eeg analysis library
import scipy.io
#import torch.nn as nn
#import torch
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tqdm import tqdm

In [5]:
#X = np.load('C:\\Users\\47575909\\Desktop\\X_PCA.npy')
#y = np.load('C:\\Users\\47575909\\Desktop\\y_PCA.npy')

In [3]:
#X.shape

NameError: name 'X' is not defined

In [7]:
#ACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
#PRUEBO CON MENOS DATOS
#ACA SUBO LOS ARCHIVOS DE TFRECORD
TFrecord_directory = 'C:\\Users\\47575909\\Desktop\\data_para_predict' 

#lista de archivos TFRecord en la carpeta
filenames = [os.path.join(TFrecord_directory, f) for f in os.listdir(TFrecord_directory) if f.endswith('.tfrecord')]

#Hago un dataset de TensorFlow a partir de los archivos TFRecord
dataset = tf.data.TFRecordDataset(filenames)

#función para parsear los datos de TFRecord
def parse_tfrecord_fn(example):
    feature_description = {
        'grpno': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'path': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example

parsed_dataset = dataset.map(parse_tfrecord_fn)

#batching para argar los datos en lotes
batch_size = 2  #puedo cambiar el tamaño (en la PC del colegio el máximo es 16, pero capáz que le tengo que poner menos)
batched_dataset = parsed_dataset.batch(batch_size)

In [30]:
#batched_dataset

In [4]:
'''#SEPARO EN X E Y
X = []
y = []

#itero los lotes
for batch_data in batched_dataset:
    paths = batch_data['path'].numpy()
    labels = batch_data['label'].numpy()
    
    #cargo los scaleograms desde los archivos .npy
    batch_X = []
    for path in paths:
        spectrogram = np.load(path.decode('utf-8'))
        batch_X.append(spectrogram)
    
#//////NUEVO

    #primero tengo que convertirlo en un archivo numpy porque sino no quiere
    batch_X = np.array(batch_X)
    
    X_shape = batch_X.shape
    shape_0 = X_shape[0]

    # Aplanar
    batch_X = batch_X.reshape(shape_0, -1)
    
    #NORMALIZO X en rango [-1, 1] para que el cero quede en el centro
    min_val = np.min(batch_X)
    max_val = np.max(batch_X)

    X = -1 + 2 * (batch_X - min_val) / (max_val - min_val)
    
    variance_to_keep = 0.95  #me quedo con el 95% de la varianza, esto es flexible (mientras más porcentaje, mayor va  quedar la dimensionalidad de mi X)

    pca = PCA()
    batch_X = pca.fit_transform(batch_X)  #aplico pca para los datos escalados

    #Encuentro el número mínimo de componentes principales para alcanzar la varianza deseada
    cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
    #('n_components_to_keep' contendrá el número mínimo de componentes principales necesarios para explicar al menos el 95% de la varianza en los datos.)
    n_components_to_keep = np.argmax(cumulative_variance_ratio >= variance_to_keep) + 1 #chona me dijo que me deberían quedar por ejemplo 1000 atributos

    #Selecciono la cant de componentes principales que me salió
    batch_X = batch_X[:, :n_components_to_keep]
    
#//////NUEVO
    
    #agrego los scaleograms y labels a las listas X y y
    X.concatenate(batch_X)
    y.extend(labels)


#convierto X e y a formato .npy
X = np.concatenate(X, axis=0)
y = np.array(y)

print('Ya se separó todo!')
print('Salió todo bien, tranquina :))')'''

"#SEPARO EN X E Y\nX = []\ny = []\n\n#itero los lotes\nfor batch_data in batched_dataset:\n    paths = batch_data['path'].numpy()\n    labels = batch_data['label'].numpy()\n    \n    #cargo los scaleograms desde los archivos .npy\n    batch_X = []\n    for path in paths:\n        spectrogram = np.load(path.decode('utf-8'))\n        batch_X.append(spectrogram)\n    \n#//////NUEVO\n\n    #primero tengo que convertirlo en un archivo numpy porque sino no quiere\n    batch_X = np.array(batch_X)\n    \n    X_shape = batch_X.shape\n    shape_0 = X_shape[0]\n\n    # Aplanar\n    batch_X = batch_X.reshape(shape_0, -1)\n    \n    #NORMALIZO X en rango [-1, 1] para que el cero quede en el centro\n    min_val = np.min(batch_X)\n    max_val = np.max(batch_X)\n\n    X = -1 + 2 * (batch_X - min_val) / (max_val - min_val)\n    \n    variance_to_keep = 0.95  #me quedo con el 95% de la varianza, esto es flexible (mientras más porcentaje, mayor va  quedar la dimensionalidad de mi X)\n\n    pca = PCA()\n 

In [6]:
#bd_iter = iter(batched_dataset)
#batch_data = next(bd_iter)

In [7]:
#batch_data

{'grpno': <tf.Tensor: shape=(2,), dtype=int64, numpy=array([10, 10], dtype=int64)>,
 'label': <tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 0], dtype=int64)>,
 'path': <tf.Tensor: shape=(2,), dtype=string, numpy=
 array([b'C:\\Users\\47575909\\Desktop\\scaleogram\\subvideo_11/trial_0.npy',
        b'C:\\Users\\47575909\\Desktop\\scaleogram\\subvideo_11/trial_1.npy'],
       dtype=object)>}

In [12]:
'''
paths = batch_data['path'].numpy()
labels = batch_data['label'].numpy()
    
# Cargar los scaleograms desde los archivos .npy
batch_X = []
for path in paths:
    spectrogram = np.load(path.decode('utf-8'))
    batch_X.append(spectrogram)
    
batch_X = np.array(batch_X)
        
# batch_X = batch_X.reshape((batch_X.shape[0], -1))  # Aplanar los datos
    
# NORMALIZA X en rango [-1, 1] para que el cero quede en el centro
min_val = np.min(batch_X)
max_val = np.max(batch_X)
    
batch_X = -1 + 2 * (batch_X - min_val) / (max_val - min_val)

batch_X'''

array([[[[-0.99985725, -0.99985456, -0.9998539 , ..., -0.9989402 ,
          -0.9989198 , -0.9989116 ],
         [-0.9997775 , -0.9997736 , -0.99977314, ..., -0.9983727 ,
          -0.99833775, -0.998324  ],
         [-0.99966854, -0.99966335, -0.99966353, ..., -0.99761903,
          -0.9975616 , -0.99753934],
         ...,
         [-0.994659  , -0.9946357 , -0.99461246, ..., -0.9908578 ,
          -0.9908636 , -0.9908695 ],
         [-0.96086556, -0.9608654 , -0.96086526, ..., -0.9608342 ,
          -0.96083426, -0.9608343 ],
         [-0.87473947, -0.87473947, -0.87473947, ..., -0.8747394 ,
          -0.87473947, -0.87473947]],

        [[-0.99991345, -0.9999164 , -0.9999175 , ..., -0.999667  ,
          -0.9996697 , -0.9996768 ],
         [-0.99988097, -0.99988747, -0.99989027, ..., -0.9994956 ,
          -0.99949825, -0.9995087 ],
         [-0.99984705, -0.99986124, -0.99986786, ..., -0.99927205,
          -0.999273  , -0.9992872 ],
         ...,
         [-0.995192  , -0.9951628 

In [13]:
#batch_X[0].shape

(14, 230, 384)

In [10]:
#batch_X.shape, labels.shape

((2, 1236480), (2,))

In [36]:
#np.resize(batch_X[0], (3, 224, 224))

array([[[-0.99985725, -0.99985456, -0.9998539 , ..., -0.99991643,
         -0.99991775, -0.9999196 ],
        [-0.99992204, -0.999925  , -0.9999283 , ..., -0.9999353 ,
         -0.99993604, -0.9999361 ],
        [-0.9999355 , -0.9999346 , -0.9999341 , ..., -0.99990755,
         -0.9999017 , -0.9998941 ],
        ...,
        [-0.9664635 , -0.96722597, -0.9679918 , ..., -0.99134135,
         -0.99110097, -0.9908375 ],
        [-0.9905519 , -0.9902452 , -0.98991853, ..., -0.98797005,
         -0.988031  , -0.98809224],
        [-0.9881534 , -0.98821443, -0.9882751 , ..., -0.9532691 ,
         -0.9527509 , -0.9522377 ]],

       [[-0.9517298 , -0.9512274 , -0.95073086, ..., -0.9882792 ,
         -0.98808426, -0.98788995],
        [-0.98769647, -0.98750395, -0.9873127 , ..., -0.9355713 ,
         -0.935637  , -0.93570983],
        [-0.93578964, -0.9358762 , -0.93596923, ..., -0.9767152 ,
         -0.9765628 , -0.97640944],
        ...,
        [-0.9934355 , -0.9931943 , -0.9930401 , ..., -

In [32]:
#data_1d =  np.array([features_2d.flatten() for features_2d in batch_X[0]])
#data_1d.shape

(14, 88320)

In [8]:
# Define el número máximo de componentes principales que deseas mantener para todos los lotes
#max_components = 1000  # Puedes ajustar este valor según tus necesidades

X_list = []  # Lista temporal para X
y_list = []  # Lista temporal para y


In [9]:
# Crea un objeto tqdm para rastrear el progreso
pbar = tqdm(total=len(filenames))

# Itera los lotes
for batch_data in batched_dataset:
    paths = batch_data['path'].numpy()
    labels = batch_data['label'].numpy()
    
    # Cargar los scaleograms desde los archivos .npy
    batch_xs = []
    for path in paths:
        spectrogram = np.load(path.decode('utf-8'))
        batch_xs.append(spectrogram)  # Corregir el nombre de la variable aquí
    
    # Convierte batch_x en un arreglo NumPy
    batch_xs = np.array(batch_xs)

    for (batch_x, label) in zip(batch_xs, labels):

        amm = batch_x.shape[0] * batch_x.shape[1] * batch_x.shape[2]
    
        x_flat = batch_x.reshape(amm, -1).squeeze()
        (batch_x.shape[0], batch_x.shape[1], batch_x.shape[2]), x_flat.shape
    
        multiplier = 1
    
        while 224 * 224 * 3 * multiplier < amm:
            multiplier += 1
    
        # Creo un array de ceros de 244*244*3*multiplier, mas grande que mi input
        x_extended = np.zeros((224 * 224 * 3 * multiplier))
    
        # Relleno el array de ceros con mi input, me quedan ceros de relleno
        x_extended[:amm] = x_flat
    
        # Reshapeo
        x_big_2d = x_extended.reshape(224 * 224, 3 * multiplier)
        x_big_2d.shape
    
        pca = PCA(
            n_components=3,
        )
    
        pca.fit(x_big_2d)
    
        x_big_2d_reduced = pca.transform(x_big_2d)
    
        x_big_3d_reduced = x_big_2d_reduced.reshape(224, 224, 3)
        
        # Agrega los scaleograms y labels a X_list e y_list
        X_list.append(x_big_3d_reduced)
        y_list.append(label)
    
        print(pca.explained_variance_ratio_, np.sum(pca.explained_variance_ratio_))
    
    # Actualiza la barra de progreso
    pbar.update(len(paths))  # Actualiza la barra de progreso según la cantidad de archivos en el lote

# Convierte X_list e y_list en arreglos NumPy
X = np.array(X_list)
y = np.array(y_list)

# Cierra la barra de progreso
# pbar.close()



297it [02:19,  2.13it/s]                                                                         | 0/1 [00:00<?, ?it/s]

2it [00:00, 11.65it/s]                                                                                                 

[0.98447994 0.01027934 0.00236879] 0.9971280670765491
[0.98163964 0.01222149 0.00279338] 0.996654512124952
[0.9829829  0.01131234 0.00252343] 0.9968186713155835



4it [00:00, 11.01it/s]
6it [00:00, 11.47it/s]

[0.98880257 0.0072079  0.00176042] 0.9977708876183397
[0.9895006  0.0071196  0.00147605] 0.9980962539557829
[0.98774516 0.00847514 0.00169244] 0.9979127501547903



8it [00:00, 11.56it/s]

...

[0.97560077 0.01711507 0.00389387] 0.9966097117035042
[0.97391968 0.01830009 0.00405659] 0.9962763572444442
[0.97265133 0.01942743 0.00424961] 0.9963283698348192



296it [00:25, 11.98it/s]

[0.97179023 0.0200362  0.00451961] 0.9963460303814681
[0.97059032 0.02084053 0.00466662] 0.99609745960426
[0.97275042 0.01934273 0.0044835 ] 0.9965766454401527



298it [00:25, 12.01it/s]

[0.97617441 0.01656554 0.00402867] 0.9967686150402755
[0.97932399 0.01410182 0.00341593] 0.9968417425795524


In [8]:
X.shape, y.shape

((1790, 3, 244, 244), (1790,))

In [34]:
'''variance_to_keep = 0.95  # Me quedo con el 95% de la varianza
n_components = 14
pca = PCA(n_components=n_components)

data_reduced = pca.fit_transform(data_1d)
data_1d.shape, data_reduced.shape'''

((14, 88320), (14, 14))

In [4]:
'''VIEJO BUENO
# Crea un objeto tqdm para rastrear el progreso
pbar = tqdm(total=len(filenames))

# Itera los lotes
for batch_data in batched_dataset:
    paths = batch_data['path'].numpy()
    labels = batch_data['label'].numpy()
    
    # Cargar los scaleograms desde los archivos .npy
    batch_X = []
    for path in paths:
        spectrogram = np.load(path.decode('utf-8'))
        batch_X.append(spectrogram)
    
    # Convierte batch_X en un arreglo NumPy
    batch_X = np.array(batch_X)
    
    batch_X = batch_X.reshape((batch_X.shape[0], -1))  # Aplanar los datos
    
    # NORMALIZA X en rango [-1, 1] para que el cero quede en el centro
    min_val = np.min(batch_X)
    max_val = np.max(batch_X)

    batch_X = -1 + 2 * (batch_X - min_val) / (max_val - min_val)
    
    variance_to_keep = 0.95  # Me quedo con el 95% de la varianza
    n_components = 3
    pca = PCA(n_components=n_components)
    batch_X = pca.fit_transform(batch_X)  # Aplica PCA para los datos escalados
    

    # Asegúrate de que todos los lotes tengan la misma cantidad de componentes principales
    n_components_to_keep = min(max_components, batch_X.shape[1])
    if batch_X.shape[1] < max_components:
        # Rellena con ceros las columnas faltantes si es necesario
        zeros_to_add = max_components - batch_X.shape[1]
        zeros = np.zeros((batch_X.shape[0], zeros_to_add))
        batch_X = np.hstack((batch_X, zeros))
    else:
        batch_X = batch_X[:, :n_components_to_keep]

    # Agrega los scaleograms y labels a X_list e y_list
    X_list.append(batch_X)
    y_list.extend(labels)

    # Actualiza la barra de progreso
    pbar.update(len(paths))  # Actualiza la barra de progreso según la cantidad de archivos en el lote

# Convierte X_list e y_list en arreglos NumPy
X = np.concatenate(X_list, axis=0)
y = np.array(y_list)

# Cierra la barra de progreso
pbar.close()'''

12488it [05:03, 34.58it/s]                                                                                             

MemoryError: Unable to allocate 9.43 MiB for an array with shape (2, 1236480) and data type float32

In [10]:
#LOS GUARDO EN MI COMPUTADORA
#np.save('X.npy', X) está guardada normalizada en rango [0, 1], habría que cambiar eso
np.save('X_predict_epilepsy.npy', X)
#np.save('y_predict_normal.npy', y)


299it [00:40, 12.01it/s]

In [6]:
X_shape = X.shape
shape_0 = X_shape[0]
X_shape

(29588, 1000)

In [7]:
# Aplanar tus datos
X = X.reshape(shape_0, -1)

In [8]:
#NORMALIZO X en rango [-1, 1] para que el cero quede en el centro
min_val = np.min(X)
max_val = np.max(X)

X = -1 + 2 * (X - min_val) / (max_val - min_val)

#X = (X - X.min()) / (X.max() - X.min()) [0, 1]

In [9]:
X.shape

(29588, 1000)

In [10]:
'''from sklearn.decomposition import PCA

variance_to_keep = 0.95  #me quedo con el 95% de la varianza, esto es flexible (mientras más porcentaje, mayor va  quedar la dimensionalidad de mi X)

pca = PCA()
X = pca.fit_transform(X)  #aplico pca para los datos escalados

#Encuentro el número mínimo de componentes principales para alcanzar la varianza deseada
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
#('n_components_to_keep' contendrá el número mínimo de componentes principales necesarios para explicar al menos el 95% de la varianza en los datos.)
n_components_to_keep = np.argmax(cumulative_variance_ratio >= variance_to_keep) + 1 #chona me dijo que me deberían quedar por ejemplo 1000 atributos

#Selecciono la cant de componentes principales que me salió
X = X[:, :n_components_to_keep]'''

"from sklearn.decomposition import PCA\n\nvariance_to_keep = 0.95  #me quedo con el 95% de la varianza, esto es flexible (mientras más porcentaje, mayor va  quedar la dimensionalidad de mi X)\n\npca = PCA()\nX = pca.fit_transform(X)  #aplico pca para los datos escalados\n\n#Encuentro el número mínimo de componentes principales para alcanzar la varianza deseada\ncumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)\n#('n_components_to_keep' contendrá el número mínimo de componentes principales necesarios para explicar al menos el 95% de la varianza en los datos.)\nn_components_to_keep = np.argmax(cumulative_variance_ratio >= variance_to_keep) + 1 #chona me dijo que me deberían quedar por ejemplo 1000 atributos\n\n#Selecciono la cant de componentes principales que me salió\nX = X[:, :n_components_to_keep]"

In [11]:
'''print("Me quedé con " + str(n_components_to_keep) + " componentes principales")
X.shape'''

'print("Me quedé con " + str(n_components_to_keep) + " componentes principales")\nX.shape'

In [12]:
'''print("Varianza explicada por cada componente principal:", pca.explained_variance_ratio_)'''

'print("Varianza explicada por cada componente principal:", pca.explained_variance_ratio_)'

In [ ]:
#no se si lo que hice está bien, chona me dijo que usara PaCMAP pero no lo entiendo, y acá encontre que con sklearn medio que se puede hacer asi que :)

In [ ]:
#hago una función?